In [1]:
import os
import rsnapsim as rss
import numpy as np
print('numpy version =', np.version.version)
print('rsnapsim version =',rss.__version__)
from sys import platform
import os
from skimage import io ; from skimage.io import imread; from skimage.measure import find_contours
from random import randrange
import os; from os import listdir; from os.path import isfile, join
import pandas as pd
import os; from os import listdir; from os.path import isfile, join
import re
import shutil
from random import randrange
import tifffile
# Importing rSNAPsim_IP
cwd = os.getcwd(); os.chdir('../../rsnaped'); import rsnaped as rsp ; os.chdir(cwd) # return to the working directory
import ipywidgets as widgets ; from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, Layout, GridspecLayout ; from IPython.display import Image

numpy version = 1.20.1
rsnapsim version = 0.0.18


In [2]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.rcParams['figure.dpi'] = 120
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['font.size'] = 15
plt.rcParams['axes.grid'] = False

In [3]:
# gene files
gene_file_h2b = '../../DataBases/gene_files/H2B_withTags.txt'
gene_file_bact = '../../DataBases/gene_files/Bactin_withTags.txt'

In [4]:
# initial video
empty_cells_directory_name = '../../DataBases/videos_for_sim_cell'

In [5]:
list_files_names = sorted([f for f in listdir(empty_cells_directory_name) if isfile(join(empty_cells_directory_name, f)) and ('.tif') in f], key=str.lower)  # reading all tif files in the folder
list_files_names.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order
path_files = [ empty_cells_directory_name+'/'+f for f in list_files_names ] # creating the complete path for each file
num_cell_shapes = len(path_files)
sel_shape = randrange(num_cell_shapes)
video_path = path_files[sel_shape]

In [6]:
# Inputs
inial_video = io.imread(video_path) # video with empty cell

list_gene_sequences = [gene_file_h2b, gene_file_bact] # path to gene sequences
list_number_spots = [10, 20] # list of integers, where each element represents the number of spots
list_target_channels = [1, 1] # channel where the simulated spots will be located. 
list_diffusion_coefficients =[0.1, 0.5] # diffusion coefficients for each gene
list_label_names = ['1','2'] # list of strings used to generate a classification field in the output data frame

simulation_time_in_sec = 30
step_size_in_sec =1

save_as_tif = 1 # option to save the simulated video
save_dataframe = 1 # option to save the simulation output as a dataframe in format csv. 
saved_file_name = 'simulated_cell' # if the video or dataframe are save, this variable assigns the name to the files

In [7]:
class SimulatedCellMultiplexing ():
    def __init__(self,inial_video,list_gene_sequences,list_number_spots,list_target_channels,list_diffusion_coefficients,list_label_names,simulation_time_in_sec,step_size_in_sec=1,save_as_tif=0, save_dataframe=1, saved_file_name='temp',create_temp_folder = True):
        self.inial_video = inial_video
        self.list_gene_sequences = list_gene_sequences
        self.list_number_spots = list_number_spots
        self.list_target_channels = list_target_channels
        self.list_diffusion_coefficients = list_diffusion_coefficients
        self.list_label_names = list_label_names
        self.simulation_time_in_sec = simulation_time_in_sec
        self.step_size_in_sec = step_size_in_sec
        self.number_genes = len(list_gene_sequences)
        self.save_as_tif = save_as_tif
        self.save_dataframe = save_dataframe
        self.saved_file_name = saved_file_name
        self.create_temp_folder = create_temp_folder
        
    def make_simulation (self):
        # FUNCTION THAT RUNS THE SSA IN rSNAPsim
        def rsnapsim_ssa(gene_file,ke =3,ki=0.033,frames=100,n_traj=20):
            poi_strs, poi_objs, tagged_pois,raw_seq = rss.seqmanip.open_seq_file(gene_file)
            gene_obj = tagged_pois['1'][0]
            gene_obj.ke = ke
            rss.solver.protein = gene_obj #pass the protein object
            t_burnin = 1000
            t = np.linspace(0,t_burnin+frames,t_burnin+frames+1)    # ask Will how to pass the step_size
            ssa_solution = rss.solver.solve_ssa(gene_obj.kelong, t, ki=ki, kt = ke, low_memory=False,record_stats=False,n_traj=n_traj)    
            time = ssa_solution.time[t_burnin:-1]
            time= time-t_burnin
            ssa_int =  ssa_solution.intensity_vec[0,t_burnin:-1,:].T
            return ssa_solution.time[t_burnin:-1], ssa_int        
        # Wrapper for the simulated cell
        def wrapper_SimulatedCell (base_video,video_for_mask=None, ssa=None, target_channel=1, diffusion_coefficient=0.05, step_size=1,spot_size=5, spot_sigma=2,intensity_calculation_method='disk_donut'):
            if target_channel ==0:
                ignore_ch0=0; ignore_ch1=1; ignore_ch2=1
            elif target_channel ==1:
                ignore_ch0=0; ignore_ch1=0; ignore_ch2=1
            elif target_channel ==2:
                ignore_ch0=0; ignore_ch1=1; ignore_ch2=0
            number_spots_per_cell = ssa.shape[0]
            simulation_time_in_sec =  ssa.shape[1]  # THIS NEEDS TO BE UPDATED TO ALLOW THE USER TO GIVE DIFFERENT STEP_SIZE
            tensor_video , _ , _, _, _, DataFrame_particles_intensities = rsp.SimulatedCell( base_video=base_video,video_for_mask=video_for_mask, number_spots = number_spots_per_cell, number_frames=simulation_time_in_sec, step_size=step_size_in_sec, diffusion_coefficient =diffusion_coefficient, simulated_trajectories_ch0=None, size_spot_ch0=spot_size, spot_sigma_ch0=spot_sigma, simulated_trajectories_ch1=ssa, size_spot_ch1=spot_size, spot_sigma_ch1=spot_sigma, simulated_trajectories_ch2=ssa, size_spot_ch2=spot_size, spot_sigma_ch2=spot_sigma, ignore_ch0=ignore_ch0,ignore_ch1=ignore_ch1, ignore_ch2=ignore_ch2,save_as_tif_uint8=0,save_as_tif =0,save_as_gif=0, save_dataframe=0, saved_file_name=None,create_temp_folder = False,intensity_calculation_method=intensity_calculation_method).make_simulation()      
            return tensor_video, DataFrame_particles_intensities   # [cell_number, particle, frame, red_int_mean, green_int_mean, blue_int_mean, red_int_std, green_int_std, blue_int_std, x, y].     
        # Runs the SSA and the simulated cell functions
        list_DataFrame_particles_intensities= []
        for i in range(0,self.number_genes):
            _ , ssa_solution = rsnapsim_ssa(self.list_gene_sequences[i],ke =3, ki=0.033,frames=self.simulation_time_in_sec,n_traj=self.list_number_spots[i])
            if i == 0 :
                tensor_video , DataFrame_particles_intensities = wrapper_SimulatedCell(self.inial_video, video_for_mask = self.inial_video, ssa=ssa_solution, target_channel=self.list_target_channels[i], diffusion_coefficient=self.list_diffusion_coefficients[i])        
            else:
                tensor_video , DataFrame_particles_intensities = wrapper_SimulatedCell(tensor_video, video_for_mask = self.inial_video, ssa=ssa_solution, target_channel=self.list_target_channels[i], diffusion_coefficient=self.list_diffusion_coefficients[i])        
            list_DataFrame_particles_intensities.append(DataFrame_particles_intensities)
        # Adding a classification column to all dataframes
        for i in range(0,self.number_genes):
            classification = self.list_label_names[i]
            list_DataFrame_particles_intensities[i]['Classification'] = classification
        # Merging multiple dataframes in a single one.
        dataframe_simulated_cell = pd.concat(list_DataFrame_particles_intensities)
        
        # saving the simulated video and data frame
        if self.save_as_tif==1:
            if self.create_temp_folder == True:
                save_to_path = 'temp/'
                if not os.path.exists(save_to_path):
                    os.makedirs(save_to_path)
                print ("The output is saved in the directory: ./" , save_to_path[0:-1])
            else:
                save_to_path=''
            tifffile.imwrite(save_to_path+self.saved_file_name+'.tif', tensor_video)
        if self.save_dataframe==1:
            if self.create_temp_folder == True:
                save_to_path = 'temp/'
                if not os.path.exists(save_to_path):
                    os.makedirs(save_to_path)
                print ("The output is saved in the directory: ./" , save_to_path[0:-1])
            else:
                save_to_path=''
            dataframe_simulated_cell.to_csv(save_to_path+self.saved_file_name +'_df'+ '.csv', index = True)
        
        return tensor_video, dataframe_simulated_cell

In [8]:
tensor_video, dataframe_simulated_cell = SimulatedCellMultiplexing(inial_video,list_gene_sequences,list_number_spots,list_target_channels,list_diffusion_coefficients,list_label_names,simulation_time_in_sec,save_as_tif,save_dataframe,saved_file_name).make_simulation()

The output is saved in the directory: ./ temp


In [9]:
# Plot only selected trajectories
plt.rcParams['figure.figsize'] = [5,5]
controls_pr, output_pr = rsp.VisualizerVideo(tensor_video,normalize=False).make_video_app()
display(VBox([controls_pr, output_pr]));

In [10]:
dataframe_simulated_cell

,cell_number,particle,frame,red_int_mean,green_int_mean,blue_int_mean,red_int_std,green_int_std,blue_int_std,x,y,Classification
0,0,0,0,46.158573,9.075714,0.0,36.601813,40.705156,NaN,297.0,137.0,gene_1
1,0,0,1,33.755714,25.147142,0.0,59.884553,32.743532,NaN,297.0,137.0,gene_1
2,0,0,2,34.292858,8.680000,0.0,44.889912,26.355344,NaN,297.0,137.0,gene_1
3,0,0,3,24.624287,11.608571,0.0,55.163378,25.980762,NaN,297.0,137.0,gene_1
4,0,0,4,37.468571,26.584286,0.0,56.687358,25.510585,NaN,297.0,137.0,gene_1
...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,19,25,79.919998,95.462860,0.0,52.553676,19.807362,NaN,218.0,348.0,gene_2
596,0,19,26,173.461426,103.092857,0.0,56.789022,29.567690,NaN,219.0,348.0,gene_2
597,0,19,27,79.384285,96.128571,0.0,54.268478,26.035199,NaN,219.0,348.0,gene_2
598,0,19,28,169.652863,93.842857,0.0,47.816417,17.704166,NaN,219.0,347.0,gene_2
